# Modelo Avanzado Tabular V2.0

## Importación de datos, imputación de NaN y features para modelo

Importación de librerías necesarias 

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

1) Carga 

In [3]:
ruta_dataset = "../data/processed/dataset_final_enriquecido.csv"
df = pd.read_csv(ruta_dataset)
df['date'] = pd.to_datetime(df['date'])
print(f"✅ Dataset enriquecido cargado. Dimensiones: {df.shape}")

✅ Dataset enriquecido cargado. Dimensiones: (37399, 25)


2) Limpieza final

In [ ]:
# Lógica de imputación para asegurar que no haya NaN
mapeo_columnas = {
    'TMAX': 'om_temperature_2m_max', 'TMIN': 'om_temperature_2m_min',
    'TAVG': 'om_apparent_temperature_mean', 'PRCP': 'om_precipitation_sum'
}
for col_noaa, col_om in mapeo_columnas.items():
    df[col_noaa] = df[col_noaa].fillna(df[col_om])
df = df.interpolate(method='time').fillna(0)
print("✅ Limpieza de NaN finalizada.")

# INGENIERÍA DE CARACTERÍSTICAS
df['mes'] = df['date'].dt.month
df['dia_del_año'] = df['date'].dt.dayofyear
df['rango_temp_diario'] = df['TMAX'] - df['TMIN']
print("✅ Nuevas características ('mes', 'rango_temp_diario', etc.) creadas.")


3) Selección de Características y Objetivo

In [ ]:
y = df['granizo']
columnas_a_excluir = [
    'date', 'station_name', 'granizo', 'latitude', 'longitude',
    'om_temperature_2m_max', 'om_temperature_2m_min', 
    'om_apparent_temperature_mean', 'om_precipitation_sum'
]
X = df.drop(columns=columnas_a_excluir)
print(f"\n✅ Datos separados. Usaremos {X.shape[1]} características para predecir.")


4) División de Entrenamiento y Prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("\n✅ Datos divididos en conjuntos de entrenamiento y prueba.")

5) Escalado de Características 

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("✅ Datos escalados exitosamente.")

## Construir y Entrenar el Modelo Avanzado (V2.0)

In [ ]:
Importacion de librerias

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import class_weight
import numpy as np

1) Calcular Pesos para Clases Desbalanceadas 
   Este paso sigue siendo crucial para que el modelo preste atención a los pocos casos de granizo.

In [ ]:
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))
print(f"Pesos de clase calculados: {class_weights_dict}")


2) Definir la Arquitectura del Modelo Avanzado
   Haremos la red un poco más "profunda" y "ancha" que nuestro modelo base, para que tenga más capacidad de aprender de las nuevas y complejas características.

In [ ]:
modelo_avanzado = keras.Sequential([
    # Capa de entrada (el shape ahora es mayor por las nuevas features)
    keras.layers.Dense(32, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    # Dos capas ocultas
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    # Capa de salida para la probabilidad
    keras.layers.Dense(1, activation='sigmoid')
])

3) Compilar el Modelo 

In [ ]:
modelo_avanzado.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

print("\nModelo Avanzado construido y compilado. Resumen:")
modelo_avanzado.summary()

4) Entrenar el Modelo
   Este es el paso que consume más recursos y se beneficia de la GPU de Colab.


In [ ]:
print("\nIniciando entrenamiento del Modelo Avanzado...")

historial_avanzado = modelo_avanzado.fit(
    X_train_scaled,
    y_train,
    epochs=50, # Aumentamos un poco las épocas de entrenamiento
    validation_data=(X_test_scaled, y_test),
    class_weight=class_weights_dict,
    verbose=1
)

print("\n✅ Entrenamiento finalizado.")

## Análisis de resultados

## Conclusiones

Por razones de eficiencia y entrenamiento esta notebook es simplemente de demostración, las últimas celdas de entrenamientos son procesadas en Google Colab.
Igualmente, para seguir con la investigación se adjuntará el link.

Link:
